In [1]:
import pandas as pd
import pickle as pickle
# from nltk.corpus import stopwords

In [3]:
sw=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'or', 'because', 'as', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'above', 'below', 'to', 'from', 'in', 'out', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',  'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn',  'hasn', 'haven',  'isn',  'ma', 'mightn',  'mustn',  'needn', 'shan']

In [4]:
def preprocess_fulldata(fileyear):
    with (open("data/full_data_"+fileyear+".pkl", "rb")) as openfile:
        data=pickle.load(openfile)
    groups=vote.groupby('natural_id')
    bills=groups.first().reset_index()
    bills['clean_text']=bills['summary'].str.lower()
    word_series = bills['clean_text']
    word_series = word_series.str.replace(r'[^A-Za-z]',' ')#all except alphabet
#     sw = stopwords.words('english')
    word_series.fillna(value='', inplace=True)
#     word_series = word_series.apply(lambda row:' '.join([oovDict.get(i, i) for i in row.split()]))
    word_series=word_series.apply(lambda row:' '.join([w for w in row.split() if len(w)>1] ))
    bills['tokenized_text']=word_series.apply(lambda row: [word for word in row.split() if word not in sw])
    
    bills['clip_text']=bills['tokenized_text'].map(lambda x:x[0:399])
    bills=bills[['natural_id','clip_text']]
       
    vote_processed=pd.merge(vote,bills,on='natural_id',how='inner')
    vote_processed.rename(columns={'clip_text':'clip_summary'},inplace=True) 
    with (open("data/full_data_"+fileyear+"_processed.pkl", "wb")) as openfile:
        pickle.dump(vote_processed,openfile)

In [7]:
preprocess_fulldata("20052012")
preprocess_fulldata("20152016")
preprocess_fulldata("20132014")

In [6]:
def preprocess_full_text(fileyear,clip_len=2000):
    with (open("bill_text_"+fileyear+".pkl", "rb")) as openfile:
        bill_text=pickle.load(openfile)
    bill_text['clean_text']=bill_text['full_text'].str.lower()
    word_series = bill_text['clean_text']
    word_series = word_series.str.replace(r'[^A-Za-z]', ' ')

    sw = stopwords.words('english')
    bill_text['tokenized_text']=word_series.apply(lambda row: [word for word in row.split() if word not in sw])

    bill_text['clip_text']=bill_text['tokenized_text'].map(lambda x:x[0:clip_len-1])

    bill_text_processed=bill_text.drop(['clean_text','tokenized_text','full_text'],axis=1)
    bill_text_processed.rename(columns={'clip_text':'text'},inplace=True)  
    with (open("bill_text_"+fileyear+"_processed.pkl", "wb")) as openfile:
        pickle.dump(bill_text_processed,openfile)


In [1]:
preprocess_fulltext("20052012")
preprocess_fulltext("20152016")
preprocess_fulltext("20132014")

In [2]:
#Some preliminary exploratory analysis
# len(data['natural_id'].unique())
# data['natural_id'][0]
# data['summary'][0]
# data['summary'][bills['natural_id']=='20132014r_HRES 590']
# data.loc[0,'summary']
# data=data[['natural_id','summary']]
# data.head()
# data.info()
# for name, group in groups:
#     print(name)
#     print(group)
# sum(data.vote==1.0)/len(data)#68.32 for 2005-12, 65.92 for 13-14, 61.07 for 15-16

# yes_percent=groups['vote'].mean()*100
# yes_percent.head()
# print(yes_percent.describe())
# sum(yes_percent<99)#all
# counts=word_series.map(lambda x:len(x))
# counts.idxmax()
# bill_text_processed.loc[279,'summary']
# counts.quantile(0.55)
# data.loc[5,'summary']
# data=pd.merge(vote,bill_text_processed)
# counts=word_series.map(lambda x:len(x))
# counts.max()
# vote_processed.info()
# data.info()
# bills.info()
# print(bill_text.head())
# #print(bill_text.info())
# count=bill_text['full_text'].str.count(":")
# #bill_text['clean_text'].head()
# #bill_text.loc[359] empty no text
# counts=bill_text['tokenized_text'].map(lambda x:len(x))
# #sum(counts>2000) #op 721
# #counts.len()
# #counts.quantile(0.999)
# #count=2000
# #
# print(counts.min())
# print(count.idxmin())
# print(bill_text.iloc[count.idxmin()])
# #print(counts[counts.idxmin])
# #print(counts.idxmin())
# #counts.describe()
# #bill_text.iloc[1568]['tokenized_text']
# #print(counts.head())